In [ ]:
!pip install streamlit pyngrok


In [ ]:
import pandas as pd

# Load and prepare data
df = pd.read_csv("/content/temp_solar_data.csv")
df['time'] = pd.to_datetime(df['Timestamp'])
df = df.drop(columns='Timestamp')


df['season'] = df['time'].dt.month % 12 // 3 + 1
season_map = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
df['season'] = df['season'].map(season_map)

df['date'] = df['time'].dt.date
df['hour'] = df['time'].dt.hour
df['year'] = df['time'].dt.year


df.to_csv("processed_data.csv", index=False)


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

# --- Load and process data ---
df = pd.read_csv("processed_data.csv")
df['date'] = pd.to_datetime(df['date'])
df['time'] = pd.to_datetime(df['time'])
df['year'] = df['date'].dt.year
df['hour'] = df['time'].dt.hour
df['season'] = df['time'].dt.month % 12 // 3 + 1
season_map = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
df['season'] = df['season'].map(season_map)

# --- UI ---
st.set_page_config(layout="wide")
st.title(" Solar Radiation &  Temperature Dashboard")

# Sidebar
season_choice = st.sidebar.selectbox("Select Season", sorted(df['season'].dropna().unique()))
season_df = df[df['season'] == season_choice]
available_years = sorted(season_df['year'].unique())
selected_year = st.sidebar.selectbox("Select Year", available_years + ["All Years (Avg)"])

if selected_year != "All Years (Avg)":
    plot_df = season_df[season_df['year'] == int(selected_year)]
    label_note = f"{season_choice} - {selected_year}"
else:
    plot_df = season_df.copy()
    label_note = f"{season_choice} - All Years (Avg)"

st.markdown(f"##  {label_note}")

# ==========  Static Curve for All Years ==========
if selected_year == "All Years (Avg)":
    st.markdown("###  Smoothed Average Profiles (All Years)")

    avg_df = plot_df.groupby(['date', 'hour'])[['Temperature', 'GHI', 'DHI', 'DNI']].mean().reset_index()
    avg_hourly = avg_df.groupby('hour')[['Temperature', 'GHI', 'DHI', 'DNI']].mean().rolling(3, center=True).mean().reset_index()

    col1, col2 = st.columns(2)

    with col1:
        st.subheader(" Avg Solar Radiation")
        fig_solar_avg = px.line(
            avg_hourly, x="hour", y=["GHI", "DHI", "DNI"],
            labels={"value": "Avg Radiation (W/m²)", "hour": "Hour"},
            title="Smoothed Solar Radiation (Hourly)"
        )
        st.plotly_chart(fig_solar_avg, use_container_width=True)

    with col2:
        st.subheader(" Avg Temperature")
        fig_temp_avg = px.line(
            avg_hourly, x="hour", y="Temperature",
            labels={"Temperature": "°C", "hour": "Hour"},
            title="Smoothed Temperature (Hourly)"
        )
        st.plotly_chart(fig_temp_avg, use_container_width=True)

# ==========  Animated View for Specific Years ==========
else:
    st.markdown("###  Daily Animation of Solar & Temperature Profiles")

    animation_df = plot_df.copy()
    animation_df['date_str'] = animation_df['date'].astype(str)

    col1, col2 = st.columns(2)

    with col1:
        st.subheader(" Solar Radiation")
        fig_rad = px.line(
            animation_df,
            x="hour",
            y=["GHI", "DHI", "DNI"],
            animation_frame="date_str",
            range_y=[0, animation_df[["GHI", "DHI", "DNI"]].max().max() * 1.1],
            labels={"value": "Radiation (W/m²)", "hour": "Hour of Day"},
            title="Radiation Profiles (Animated by Day)"
        )
        st.plotly_chart(fig_rad, use_container_width=True)

    with col2:
        st.subheader(" Temperature")
        fig_temp = px.line(
            animation_df,
            x="hour",
            y="Temperature",
            animation_frame="date_str",
            range_y=[animation_df["Temperature"].min() - 2, animation_df["Temperature"].max() + 2],
            labels={"Temperature": "°C", "hour": "Hour of Day"},
            title="Temperature Profile (Animated by Day)"
        )
        st.plotly_chart(fig_temp, use_container_width=True)

# ==========  KPI Tabs (for all views) ==========
with st.expander(" View Summary KPIs and Analysis"):
    tab1, tab2 = st.tabs([" Averages & Heatmap", " Significant Days"])

    with tab1:
        st.subheader(" Avg Hourly Solar Radiation")
        avg_hourly = plot_df.groupby("hour")[["Temperature", "GHI", "DHI", "DNI"]].mean().reset_index()
        fig_rad_avg = px.line(
            avg_hourly,
            x="hour",
            y=["GHI", "DHI", "DNI"],
            title=f"Solar Radiation - Hourly Averages ({label_note})",
            labels={"value": "Avg Radiation (W/m²)", "hour": "Hour of Day"}
        )
        st.plotly_chart(fig_rad_avg, use_container_width=True)

        st.subheader(" Avg Hourly Temperature")
        fig_temp_avg = px.line(
            avg_hourly,
            x="hour",
            y="Temperature",
            title=f"Temperature - Hourly Average ({label_note})",
            labels={"Temperature": "°C", "hour": "Hour of Day"}
        )
        st.plotly_chart(fig_temp_avg, use_container_width=True)

        st.subheader(" GHI Heatmap by Hour & Date")
        heat_df = plot_df.groupby(['date', 'hour'])['GHI'].mean().reset_index()
        heat_pivot = heat_df.pivot(index='date', columns='hour', values='GHI').fillna(0)
        fig_heat = px.imshow(
            heat_pivot,
            labels=dict(x="Hour", y="Date", color="GHI"),
            aspect="auto",
            title=f"Heatmap - GHI Intensity by Hour & Date ({selected_year})"
        )
        st.plotly_chart(fig_heat, use_container_width=True)

    with tab2:
        st.subheader("Significant Days Summary")
        daily_stats = plot_df.groupby('date').agg({
            'GHI': ['mean', 'std'],
            'Temperature': ['min', 'max']
        }).reset_index()
        daily_stats.columns = ['date', 'ghi_mean', 'ghi_std', 'temp_min', 'temp_max']
        daily_stats['temp_range'] = daily_stats['temp_max'] - daily_stats['temp_min']

        st.markdown("#### Peak Solar Radiation Days")
        st.dataframe(daily_stats.nlargest(3, 'ghi_mean')[['date', 'ghi_mean']])

        st.markdown("#### Low Solar Radiation Days")
        st.dataframe(daily_stats.nsmallest(3, 'ghi_mean')[['date', 'ghi_mean']])

        st.markdown("#### High Temperature Swings")
        st.dataframe(daily_stats.nlargest(3, 'temp_range')[['date', 'temp_range']])

        st.markdown("#### High Solar Variability")
        st.dataframe(daily_stats.nlargest(3, 'ghi_std')[['date', 'ghi_std']])


Overwriting app.py


In [ ]:
!ngrok config add-authtoken 2vPbi8ZSfsUB0ITjrnHASQlUHtE_2nVgMVJ29Ea5nM9erq2X9

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
import time
from pyngrok import ngrok

# Kill existing streamlit if running
os.system("pkill streamlit")

# Start the streamlit app
os.system("streamlit run app.py &")
time.sleep(10)

# Create public tunnel
public_url = ngrok.connect(8501)
print("Your Streamlit app is live at:", public_url)


Your Streamlit app is live at: NgrokTunnel: "https://b5f5-34-169-8-9.ngrok-free.app" -> "http://localhost:8501"
